## Greeks Hedge

We will use Detla, Gamma and Vega hedge for mainatain option premium in sell side.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stat
import matplotlib.pyplot as plt

In [2]:
r= 0.005
tau = 27/365
tol = 1/100
N = 100
share = 100000
df = pd.read_excel('call.xlsx')

'call.xlsx' is Call option & Kospi200 prices. We brought it from KRX data service.
price data : 2021.04.15~2021.05.13

## We have 3 steps for maintain option premium with the degree of Greeks hedge.

#### 1. Decide the Call option to sell and its quantity
#### 2. Find Greeks of Call option with BSM
#### 3. Hedge using Greeks and find pay-off

In [3]:
def C435():
    list=[]
    for i in range(1,21):
        list.append(df.iloc[0,i])
    return(list)
def C440():
    list=[]
    for i in range(1,21):
        list.append(df.iloc[1,i])
    return(list)
def C445():
    list=[]
    for i in range(1,21):
        list.append(df.iloc[2,i])
    return(list)
def Kospi():
    list=[]
    for i in range(1,21):
        list.append(df.iloc[3,i])
    return(list)  

I made some functions for data sets

In [4]:
def Call_sigma(S, K, tau, r, sigma, C):
    d1 = (np.log(S/K)+(r + 0.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
    d2 = (np.log(S/K)+ (r - 0.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
    return S*stat.norm.cdf(d1) - K*np.exp(-r*tau)*stat.norm.cdf(d2) -C
def Vega (S, K, tau, r, sigma):
    d1 = (np.log(S/K)+(r + 0.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
    return S*np.sqrt(tau)*stat.norm.pdf(d1)
def Call_gamma(S,K,tau,r,sigma):
    if tau>0:
        d1 = (np.log(S/K)+(r + 0.5*sigma**2)*(tau))/(sigma*np.sqrt(tau))
        return stat.norm.pdf(d1)/(S*sigma*np.sqrt(tau))
    elif tau<=0:
        return 0
    else    :
        print('input time before T')
def Call_delta(S,K,tau,r,sigma):
    if tau>0:
        d1 = (np.log(S/K)+(r + 0.5*sigma**2)*(tau))/(sigma*np.sqrt(tau))
        return stat.norm.cdf(d1)
    elif tau<0:
        if S > K:
            return 1
        elif S==K:
            return 0.5
        elif S < K:
            return 0
        else:
            print('worng S or E')
    else:
        print('input time before T')

## Newtion Method for 43 strike price% market price

In [5]:
def newton_method(tol,N,Call_data,strike_price,asset_price):
    L = len(Call_data)
    K = np.zeros(shape = L)
    for i in range(0,L):
        K[i] = strike_price 
    volatility = np.zeros(shape = L)
    for j in range(0, L):
        volatility0 = 0.5
        i = 1
        while i < N:
            volatility[j] = volatility0 - Call_sigma(asset_price[j], K[j], tau, r, volatility0, Call_data[j])/Vega(asset_price[j], K[j], tau, r, volatility0)
            if abs(volatility[j] - volatility0) < tol:
                print (volatility[j])
                break
            else:
                i = i+1
                volatility0 = volatility[j]
                if i == N:
                    print('The method failed after N iterations, N=',N)
    return volatility

In [6]:
V = newton_method(tol,N,C435(),435,Kospi())
cash = C435()[0]*share
print('option premiums from short C435',cash)
print("greeks",Call_delta(Kospi()[0],435,tau,r,V[0]))
print(Call_gamma(Kospi()[0],435,tau,r,V[0]))
print(Vega(Kospi()[0],435,tau,r,V[0]))
print(Call_delta(Kospi()[0],440,tau,r,V[0]))
print(Call_gamma(Kospi()[0],440,tau,r,V[0]))
print(Vega(Kospi()[0],440,tau,r,V[0]))
print(Call_delta(Kospi()[0],445,tau,r,V[0]))
print(Call_gamma(Kospi()[0],445,tau,r,V[0]))
print(Vega(Kospi()[0],445,tau,r,V[0]))
greeks = np.array([[Call_gamma(Kospi()[0],440,tau,r,V[0]),Call_gamma(Kospi()[0],445,tau,r,V[0])],
                    [Vega(Kospi()[0],440,tau,r,V[0]),Vega(Kospi()[0],445,tau,r,V[0])]])
portfolio_greeks=[[share*Call_gamma(Kospi()[0],435,tau,r,V[0])],[share*Vega(Kospi()[0],435,tau,r,V[0])]]
inv = np.linalg.inv(np.round(greeks,2))
print("inv",inv)
w = np.dot(inv,portfolio_greeks)
print('need to long Calls with strike price = 440,445',w)
print(np.round(np.dot(np.round(greeks, 2), w) - portfolio_greeks))
portfolio_greeks = [[Call_delta(Kospi()[0],435,tau,r,V[0])*-share],
                     [Call_gamma(Kospi()[0],435,tau,r,V[0])*-share],
                     [Vega(Kospi()[0],435,tau,r,V[0])*-share]]
greeks = np.array([[Call_delta(Kospi()[0],440,tau,r,V[0]),Call_delta(Kospi()[0],445,tau,r,V[0])], 
                   [Call_gamma(Kospi()[0],440,tau,r,V[0]), Call_gamma(Kospi()[0],445,tau,r,V[0])], 
                   [Vega(Kospi()[0],440,tau,r,V[0]), Vega(Kospi()[0],445,tau,r,V[0])]])
print(np.round(np.dot(np.round(greeks, 2), w) + portfolio_greeks))
long_Kospi= [[10912],[0],[0]]
print(np.round(np.dot(np.round(greeks, 2), w) + portfolio_greeks + long_Kospi))
cash=cash- np.round(w[0])*C440()[0] - 53397*C445()[0] - Kospi()[0]*10912

0.1467247864452535
0.14360508728091487
0.14488806806105567
0.13290961305970753
0.1424845335286656
0.13694413003457337
0.12576319461689986
0.11779214617436502
0.12006914773987638
0.12118120883487218
0.11961163258717997
0.11262237488154224
0.09819850366797547
0.08119486632123578
0.06703433574400341
0.06496934474734181
0.059863085685157656
0.05323531680456964
0.05743513437365087
0.06472791044502534
option premiums from short C435 587000.0
greeks 0.4578142631086882
0.022976675772831127
46.682604017066915
0.3474065609192169
0.021394402966043394
43.46783893892683
0.24968366663208852
0.018392649073893604
37.36905904189686
inv [[-3.06311475e+02  1.63934426e-01]
 [ 3.56311475e+02 -1.63934426e-01]]
need to long Calls with strike price = 440,445 [[61486.64484479]
 [53396.73401937]]
[[-0.]
 [-0.]]
[[-10912.]
 [    -0.]
 [    -0.]]
[[ 0.]
 [-0.]
 [-0.]]


### We asume 5 scenarios.

#### 1. Use for Delta hedge
#### 2. Use for Delta and Gamma hedge
#### 3. Use for Delta, Gamma and Vega hedge
#### 4, Deposit in Bank
#### 5. Invest in Stocks

In [7]:
#scenario 1
profit_portfolio1 = cash*np.exp(-r*tau) + Kospi()[0]*10912
#scenario 2
Asset_price = 442.5
profit_portfolio2 = cash*np.exp(r*tau) + Asset_price*10912 - (Asset_price-435)*share
#scenario 3
Asset_price = 447.5
profit_portfolio3 = (cash*np.exp(r*tau) + Asset_price*10912 - 
                     (Asset_price-435)*share + (Asset_price-440)*np.round(w[0]))
#scenario 4
Asset_price = 452.5
profit_portfolio4 = (cash*np.exp(r*tau) + Asset_price*10912 - 
                     (Asset_price-435)*share + 
                     (Asset_price-440)*np.round(w[0]) + (Asset_price-440)*np.round(w[1]))
print("profit from Kospi data",profit_portfolio1)
print("porfit of portfolio If Kospi = 442.5",profit_portfolio2)
print("porfit of portfolio If Kospi = 447.5",profit_portfolio3)
print("porfit of portfolio If Kospi = 452.5",profit_portfolio4)

profit from Kospi data [193344.8606792]
porfit of portfolio If Kospi = 442.5 [-452631.66031089]
porfit of portfolio If Kospi = 447.5 [-436919.16031089]
porfit of portfolio If Kospi = 452.5 [92538.33968911]
